### Here, I will try to apply bayesian linear regression (http://pyro.ai/examples/bayesian_regression.html) to logistic regression

In [4]:
import numpy as np
import scipy.special as ssp
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.distributions.constraints as constraints

from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

import pyro
import pyro.distributions as dist

from pyro.nn import PyroModule
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam, SGD

pyro.enable_validation(True)
torch.set_default_dtype(torch.double) # this was necessary on the CPU

In [33]:
def build_logistic_dataset(N, p=1, noise_std=0.01):
    X = np.random.randn(N, p)
    
    w = np.random.randn(p)
    w += 2 * np.sign(w)

    y = np.round(ssp.expit(np.matmul(X, w) 
                           + np.repeat(1, N) 
                           + np.random.normal(0, noise_std, size=N)))
    y = y.reshape(N, 1)
    X, y = torch.tensor(X).type(torch.Tensor), torch.tensor(y).type(torch.Tensor)
    data = torch.cat((X, y), 1)
    assert data.shape == (N, p + 1)
    return X, y, w

In [133]:
from pyro.nn import PyroSample

class BayesianLogisticRegression(PyroModule):
    def __init__(self, in_features):
        super().__init__()
        self.linear = PyroModule[nn.Linear](in_features, 1)
        
        #set prior
        self.linear.weight = PyroSample(dist.Normal(0., 1.).expand([1, in_features]).to_event(2))
        self.linear.bias = PyroSample(dist.Normal(0, 10.).expand([1]).to_event(1))
        
        self.sigmoid = PyroModule[nn.Sigmoid]()

    def forward(self, x, y=None):
        
        model_output = self.sigmoid(self.linear(x)).squeeze(-1)
        
        with pyro.plate("data", x.shape[0]):
                        
            obs = pyro.sample("obs", 
                              dist.Bernoulli(probs=model_output), #is this logits or probs?
                              obs=y.squeeze())
        return model_output

In [134]:
N, p = 5000, 3
num_iterations = 1500

X_data, y_data, w = build_logistic_dataset(N, p)

In [135]:
from pyro.infer.autoguide import AutoDiagonalNormal

model = BayesianLogisticRegression(p)
guide = AutoDiagonalNormal(model)

In [136]:
from pyro.infer import SVI, Trace_ELBO

adam = pyro.optim.Adam({"lr": 0.03})
svi = SVI(model, guide, adam, loss=Trace_ELBO())

In [137]:
pyro.clear_param_store()
for j in range(num_iterations):
    # calculate the loss and take a gradient step
    loss = svi.step(X_data, y_data)
    if j % 100 == 0:
        print("[iteration %04d] loss: %.4f" % (j + 1, loss / len(data)))

[iteration 0001] loss: 1426.7853
[iteration 0101] loss: 328.6124
[iteration 0201] loss: 249.1267
[iteration 0301] loss: 204.7029
[iteration 0401] loss: 181.9491
[iteration 0501] loss: 164.6745
[iteration 0601] loss: 148.5307
[iteration 0701] loss: 144.1651
[iteration 0801] loss: 135.2724
[iteration 0901] loss: 130.3960
[iteration 1001] loss: 126.2951
[iteration 1101] loss: 124.6198
[iteration 1201] loss: 121.7056
[iteration 1301] loss: 119.7456
[iteration 1401] loss: 117.9148


In [138]:
print(w)

[ 2.80489071 -4.72236621  2.46736118]


In [139]:
guide.requires_grad_(False)

for name, value in pyro.get_param_store().items():
    print(name, pyro.param(name))

AutoDiagonalNormal.loc Parameter containing:
tensor([ 5.4775, -9.1109,  4.7039,  2.0186])
AutoDiagonalNormal.scale tensor([0.0804, 0.1095, 0.0811, 0.0848])


In [141]:
guide.quantiles([0.5])

{'linear.weight': tensor([[[ 5.4775, -9.1109,  4.7039]]]),
 'linear.bias': tensor([[2.0186]])}

In [142]:
w * 2.0186

array([ 5.66195238, -9.53256843,  4.98061528])